Parsing wikipedia
- Vyparsovanie automobilových vozidiel z wikipédii a vytvorenie služby, ktorá by podľa zadaného automobilu (podľa jeho kategórii (automobilka, rok výroby, produkcia, trieda, predchodca, typ karosérie) umožňovala vyhľadať podobné automobilové vozidlá danej kategórie

In [50]:
import bz2
import re

vehicles = []

In [51]:
class Vehicle:
    def __init__(self, name='', manufacturer=[], production='', vehicle_class='', layout=''):
        self.name = name
        self.manufacturer = manufacturer
        self.production = production
        self.vehicle_class = vehicle_class
        self.layout = layout

In [52]:
def parse_parameter_value(parameter: str, text: str) -> str:
    pattern = f"(\|){{1}}(\s)*({parameter}){{1}}(\s)*(=){{1}}(\s)*"
    value = re.sub(pattern, '', text)
    value = re.split('[&]', value)
    return value[0].replace('[', '').replace(']', '').replace('\n', '').rstrip()


def find_parameter(parameter: str, text: str):
    pattern = f"(\|){{1}}(\s)*({parameter}){{1}}(\s)*(=){{1}}(\s)*"
    return re.search(pattern, text)


def get_vehicle_parameters(vehicles: list, text: str):
    text = text.lower()
    if find_parameter('name', text):
        vehicles[-1].name = parse_parameter_value('name', text)
    if find_parameter('manufacturer', text):
        manufacturers = parse_parameter_value('manufacturer', text)
        manufacturers = re.split('[|]', manufacturers)
        vehicles[-1].manufacturer = [str(item).lstrip().rstrip() for item in manufacturers]
    if find_parameter('class', text):
        vehicle_class = parse_parameter_value('class', text)
        vehicle_class = re.split('[|/]', vehicle_class)
        vehicles[-1].vehicle_class = [str(item).lstrip().rstrip() for item in vehicle_class]
    if find_parameter('production', text):
        years = parse_parameter_value('production', text)
        years = re.findall(r'([1-2][0-9]{3})', years)
        if years:
            vehicles[-1].production = int(years[0])
    if find_parameter('layout', text):
        layout = parse_parameter_value('layout', text)
        layout = re.split('[|/,#]', layout)
        vehicles[-1].layout = [str(item).lstrip().rstrip() for item in layout]

In [53]:
if __name__ == "__main__":
    with bz2.BZ2File('dataset/enwiki-20220920-pages-meta-current10.xml-p4045403p5399366.bz2') as f:
        global vehicles
        flag = False
        for row in f:
            row = row.decode("utf-8")
            if '{{Infobox automobile' == row.lstrip().rstrip():
                flag = True
                vehicles.append(Vehicle())
                continue
            elif '}}' == row.rstrip().lstrip() and flag == True:
                flag = False

            if flag:
                get_vehicle_parameters(vehicles=vehicles, text=row)
        f.close()

front-engine, front-wheel drive layout|front-engine, front-wheel drive
['front-engine', ' front-wheel drive layout', 'front-engine', ' front-wheel drive']
fr layout
['fr layout']
fr layout
['fr layout']
fr layout
['fr layout']
front-engine, rear-wheel-drive layout|front-engine, rear-wheel-drive or front-engine, four-wheel-drive layout|four-wheel-drive
['front-engine', ' rear-wheel-drive layout', 'front-engine', ' rear-wheel-drive or front-engine', ' four-wheel-drive layout', 'four-wheel-drive']
fr layout
['fr layout']

['']
fr layout
['fr layout']
fr (front engine / rear-wheel drive|rwd)
['fr (front engine ', ' rear-wheel drive', 'rwd)']
front-engine, rear-wheel drive layout|fr layout
['front-engine', ' rear-wheel drive layout', 'fr layout']

['']
rear mid-engine, rear-wheel-drive layout|rear mid-engine, rear-wheel-drive
['rear mid-engine', ' rear-wheel-drive layout', 'rear mid-engine', ' rear-wheel-drive']
fr layout
['fr layout']
rear mid-engine, rear-wheel drive layout|rear mid-engin

In [ ]:
for vehicle in vehicles:
    print(vehicle.manufacturer)
    """ print({
        'name': vehicle.name,
        'manufacturer': vehicle.manufacturer,
        'class': vehicle.vehicle_class,
        'layout': vehicle.layout,
        'production': vehicle.production
    }) """
# print(len(vehicles))